In [ ]:
%store -r content_df # get the df with html

In [ ]:
import pandas as pd

df = content_df

## 0, hernan's function

In [ ]:
# hernan's function
import re


def clean_html(html_content):
    try:
        html_content = re.sub(
            r"<(script|style).*?>.*?</\1>", "", html_content, flags=re.DOTALL
        )  # Remove script and style elements
        html_content = re.sub(
            r'&{2,}.*?["\'>]', "", html_content
        )  # Remove JavaScript-like patterns and similar snippets Yes, it's needed, because we still had trash
        text = re.sub(
            r"<[^>]+>", "", html_content
        )  # Remove all remaining HTML tags but leaving stuff between tags
        text = re.sub(
            r"&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", "", text
        )  # Replace HTML entities with normal chars
        text = " ".join(text.split())
        return text
    except Exception as e:
        print("this is not html")
        return None

In [ ]:
df["regex"] = df["content"].apply(clean_html)

In [ ]:
df["regex"].apply(len).median()

## 1, first iteration

In [ ]:
# first iteration:
import re


def clean_html_1(html_content):
    try:
        # Remove content within script or style tags.
        html_content = re.sub(
            r"<(script|style).*?>.*?</\1>", "", html_content, flags=re.DOTALL
        )
        # Remove JavaScript-like patterns and other similar snippets.
        html_content = re.sub(r'&{2,}.*?["\'>]', "", html_content)
        # Remove all remaining HTML tags but leave the content between tags.
        text = re.sub(r"<[^>]+>", "", html_content)
        # Replace HTML entities with normal characters.
        text = re.sub(r"&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", "", text)
        # Remove content within curly braces, including the braces themselves.
        text = re.sub(r"\{.*?\}", "", text)
        # Normalize whitespace to single spaces.
        text = " ".join(text.split())
        return text
    except Exception as e:
        print("Error processing HTML: {}".format(e))
        return None

In [ ]:
df["regex1"] = df["content"].apply(clean_html_1)

In [ ]:
df["regex1"].apply(len).median()

In [ ]:
# Group by 'ascii_id_company' and aggregate the other columns
df_grouped = (
    df.drop(["content"], axis="columns")
    .groupby("ascii_id_company")
    .agg(
        {
            "src_url": lambda x: list(x),  # Convert all src_url values to a list
            "regex": lambda x: " ".join(x),  # Join all extracted text with a space
            "regex1": lambda x: " ".join(x),  # Join all extracted text with a space
        }
    )
    .reset_index()
)

df_grouped = df_grouped.rename(columns={"src_url": "url_list"})

In [ ]:
text = df_grouped.iloc[16][
    "regex1"
]  # this one index 16 is a good case to test on as it somehow still collects code

In [ ]:
import textwrap


def print_wrapped(text, width=80):
    print("\n".join(textwrap.wrap(text, width=width)))

In [ ]:
print_wrapped(text, width=120)

#### Notes:

still not removed the code from curly braces...